In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [22]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

img = load_img('data/train/cats/cat.10.jpg')
x = img_to_array(img)
x = x.reshape((1, ) + x.shape)

In [23]:
x.shape

(1, 499, 489, 3)

In [18]:
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview',
                         save_prefix='cat',
                         save_format='jpeg'):
    i += 1
    if i > 20:
        break

In [34]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_22 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 34, 34, 64)        18496     
__________

In [40]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [41]:
batch_size=16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'data/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [46]:
from keras.callbacks import ModelCheckpoint
import os

save_checkpoint = ModelCheckpoint('weights-best.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

try:
    model.load_weights('weights-best.hdf5')
except OSError:
    pass

hist = model.fit_generator(
    train_generator,
    steps_per_epoch= 2000 // batch_size,
    epochs = 50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size,
    callbacks=[save_checkpoint])


Epoch 1/50
125/125 [==============================] - 73s 582ms/step - loss: 0.5632 - acc: 0.7210 - val_loss: 0.5333 - val_acc: 0.7225

Epoch 00001: val_acc improved from -inf to 0.72250, saving model to weights-best.hdf5
Epoch 2/50
125/125 [==============================] - 84s 676ms/step - loss: 0.5442 - acc: 0.7375 - val_loss: 0.5395 - val_acc: 0.7175

Epoch 00002: val_acc did not improve from 0.72250
Epoch 3/50
125/125 [==============================] - 74s 593ms/step - loss: 0.5497 - acc: 0.7450 - val_loss: 0.5336 - val_acc: 0.7250

Epoch 00003: val_acc improved from 0.72250 to 0.72500, saving model to weights-best.hdf5
Epoch 4/50
125/125 [==============================] - 74s 588ms/step - loss: 0.5300 - acc: 0.7495 - val_loss: 0.5489 - val_acc: 0.7100

Epoch 00004: val_acc did not improve from 0.72500
Epoch 5/50
125/125 [==============================] - 72s 574ms/step - loss: 0.5379 - acc: 0.7420 - val_loss: 0.5417 - val_acc: 0.7350

Epoch 00005: val_acc improved from 0.72500 to

KeyboardInterrupt: 

In [74]:
model.load_weights('weights-best.hdf5')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('data/test', target_size=(150, 150),
    batch_size=1,
    class_mode='binary', 
    shuffle=False)

predict = model.predict_generator(test_generator, steps=len(test_generator.filenames))

Found 9 images belonging to 2 classes.


In [84]:
import numpy as np
predict

labels_to_names = {
    0: 'cat',
    1: 'dog'
}

conv_to_names = lambda x: labels_to_names[np.rint(x)]
conv_to_names(np.array(predict))

TypeError: unhashable type: 'numpy.ndarray'

In [81]:
predict

array([[0.2739436 ],
       [0.13043454],
       [0.01510532],
       [0.34122548],
       [0.38216949],
       [0.90608764],
       [0.8839372 ],
       [0.8647076 ],
       [0.9999926 ]], dtype=float32)

In [ ]:
array([[0.2739436 ],
       [0.13043454],
       [0.01510532],
       [0.34122548],
       [0.38216949],
       [0.8839372 ],
       [0.8647076 ],
       [0.9999926 ]], dtype=float32)